In [ ]:
import gc
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import random
import os
import copy
import matplotlib.pyplot as plt

from time import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F

from pytorchtools import EarlyStopping

#!pip install wandb
import wandb

# Data loading & data preprocessing

In [ ]:
# Load monthly M4 data.
# Transform the data into a lists of arrays. Each inner array represents a timeseries.
# Remove all the NaN values from the datasets.

# M4
trainset = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv')
testset = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv')
trainset.set_index('V1', inplace = True)
testset.set_index('V1', inplace = True)
# Add the testset columns behind the trainset columns
testset_merge = trainset.merge(testset, on = 'V1', how = 'inner')
# Get the data in numpy representation
trainset_np = trainset.values
testset_np = testset_merge.values
# Select all non NaN values from the trainset
trainset_clean = [x[x == x] for x in trainset_np]
# Train/validation/test --------------------------------- NBeats paper validation strategy
testset_m4m = [x[x == x] for x in testset_np]
valset_m4m = trainset_clean.copy()
trainset_m4m = [x[:-18] for x in trainset_clean]

del(trainset, testset, testset_merge, trainset_np, testset_np, trainset_clean)

# Scale independent loss functions

In [ ]:
# output = batch x fl
# target = batch x fl
# actuals_train = batch x bl

def SMAPE(output, target, actuals_train = None):
    
    abs_errors = torch.abs(target - output)
    abs_output = torch.abs(output)
    abs_target = torch.abs(target)
    loss = 200 * torch.mean(abs_errors / (abs_output.detach() + abs_target + 1e-5))
    # possibly nan values in training networks if no offset in denominator is used
    
    return loss


def MAPE(output, target, actuals_train = None):

    abs_errors = torch.abs(target - output)
    abs_target = torch.abs(target)
    loss = 100 * torch.mean(abs_errors / (abs_target + 1e-5))
    # possibly nan values in training networks if no offset in denominator is used

    return loss


def MASE(output, target, actuals_train):
    
    mask = torch.abs(actuals_train)>1e-6
    mad = torch.sum(torch.abs(actuals_train[:, 1:] - actuals_train[:, :-1]), dim = -1) / (torch.sum(mask, dim = -1) - 1)
    mad_reshaped = mad.unsqueeze(-1).repeat_interleave(target.shape[-1], dim = -1)
    loss_items = torch.mean((torch.abs(target - output)) / (mad_reshaped + 1e-5), dim = -1)
    loss_items_clamped = torch.clamp(loss_items, 0, 5)
    loss = torch.mean(loss_items_clamped)

    return loss


def MASE_m(output, target, actuals_train):
    
    mask = torch.abs(actuals_train)>1e-6
    mad = torch.sum(torch.abs(actuals_train[:, 12:] - actuals_train[:, :-12]), dim = -1) / (torch.sum(mask, dim = -1) - 12)
    mad_reshaped = mad.unsqueeze(-1).repeat_interleave(target.shape[-1], dim = -1)
    loss_items = torch.mean((torch.abs(target - output)) / (mad_reshaped + 1e-5), dim = -1)
    loss_items_clamped = torch.clamp(loss_items, 0, 5)
    loss = torch.mean(loss_items_clamped) 

    return loss


def RMSSE(output, target, actuals_train):
    
    mask = torch.abs(actuals_train)>1e-6
    msd = torch.sum((actuals_train[:, 1:] - actuals_train[:, :-1])**2, dim = -1) / (torch.sum(mask, dim = -1) - 1)
    msd_reshaped = msd.unsqueeze(-1).repeat_interleave(target.shape[-1], dim = -1)
    loss_items = torch.sqrt(torch.mean((target - output)**2 / (msd_reshaped + 1e-5), dim = -1))
    loss_items_clamped = torch.clamp(loss_items, 0, 5)
    loss = torch.mean(loss_items_clamped)
    #loss = torch.sqrt(torch.mean((target - output)**2 / msd_reshaped))

    return loss


def RMSSE_m(output, target, actuals_train):
    
    mask = torch.abs(actuals_train)>1e-6
    msd = torch.sum((actuals_train[:, 12:] - actuals_train[:, :-12])**2, dim = -1) / (torch.sum(mask, dim = -1) - 12)
    msd_reshaped = msd.unsqueeze(-1).repeat_interleave(target.shape[-1], dim = -1)
    loss_items = torch.sqrt(torch.mean((target - output)**2 / (msd_reshaped + 1e-5), dim = -1))
    loss_items_clamped = torch.clamp(loss_items, 0, 5)
    loss = torch.mean(loss_items_clamped)
    
    return loss

# NBeats models

Generic building block:

In [ ]:
class GenericNBeatsBlock(nn.Module):
    
    def __init__(self,
                 device,
                 backcast_length,
                 forecast_length,
                 hidden_layer_units, thetas_dims, 
                 share_thetas,
                 dropout = False, dropout_p = 0.0, 
                 neg_slope = 0.00):
        
        super().__init__()
        self.device = device
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length        
        if isinstance(hidden_layer_units, int):
            self.hidden_layer_units = [hidden_layer_units for FC_layer in range(4)]
        else:
            #assert(len(hidden_layer_units) == 4)
            self.hidden_layer_units = hidden_layer_units
        self.thetas_dims = thetas_dims
        self.share_thetas = share_thetas
        self.dropout = dropout
        self.dropout_p = dropout_p
        self.neg_slope = neg_slope
        
        # shared layers in block
        self.fc1 = nn.Linear(self.backcast_length,
                             self.hidden_layer_units[0])#, bias = False)
        self.fc2 = nn.Linear(self.hidden_layer_units[0], self.hidden_layer_units[1])#, bias = False)
        self.fc3 = nn.Linear(self.hidden_layer_units[1], self.hidden_layer_units[2])#, bias = False)
        self.fc4 = nn.Linear(self.hidden_layer_units[2], self.hidden_layer_units[3])#, bias = False)
        
        # do not use F.dropout as you want dropout to only affect training (not evaluation mode)
        # nn.Dropout handles this automatically
        if self.dropout:
            self.dropoutlayer = nn.Dropout(p = self.dropout_p)
        
        # task specific (backcast & forecast) layers in block
        # do not include bias - see section 3.1 - Ruben does include bias for generic blocks
        if self.share_thetas:
            self.theta_b_fc = self.theta_f_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
        else:
            self.theta_b_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
            self.theta_f_fc = nn.Linear(self.hidden_layer_units[3], self.thetas_dims)#, bias = False)
        
        # block output layers
        self.backcast_out = nn.Linear(self.thetas_dims, self.backcast_length)#, bias = False) # include bias - see section 3.3
        self.forecast_out = nn.Linear(self.thetas_dims, self.forecast_length)#, bias = False) # include bias - see section 3.3
        
        
    def forward(self, x):
        
        if self.dropout:
            h1 = F.leaky_relu(self.fc1(x.to(self.device)), negative_slope = self.neg_slope)
            h1 = self.dropoutlayer(h1)
            h2 = F.leaky_relu(self.fc2(h1), negative_slope = self.neg_slope)
            h2 = self.dropoutlayer(h2)
            h3 = F.leaky_relu(self.fc3(h2), negative_slope = self.neg_slope)
            h3 = self.dropoutlayer(h3)
            h4 = F.leaky_relu(self.fc4(h3), negative_slope = self.neg_slope)
            theta_b = F.leaky_relu(self.theta_b_fc(h4), negative_slope = self.neg_slope)
            #theta_b = self.theta_b_fc(h4)
            theta_f = F.leaky_relu(self.theta_f_fc(h4), negative_slope = self.neg_slope)
            #theta_f = self.theta_f_fc(h4)
            backcast = self.backcast_out(theta_b)
            forecast = self.forecast_out(theta_f)
        else:
            h1 = F.leaky_relu(self.fc1(x.to(self.device)), negative_slope = self.neg_slope)
            h2 = F.leaky_relu(self.fc2(h1), negative_slope = self.neg_slope)
            h3 = F.leaky_relu(self.fc3(h2), negative_slope = self.neg_slope)
            h4 = F.leaky_relu(self.fc4(h3), negative_slope = self.neg_slope)
            theta_b = F.leaky_relu(self.theta_b_fc(h4), negative_slope = self.neg_slope)
            #theta_b = self.theta_b_fc(h4)
            theta_f = F.leaky_relu(self.theta_f_fc(h4), negative_slope = self.neg_slope)
            #theta_f = self.theta_f_fc(h4)
            backcast = self.backcast_out(theta_b)
            forecast = self.forecast_out(theta_f)
            
        return backcast, forecast
    
    
    def __str__(self):
        
        block_type = type(self).__name__
        
        return f'{block_type}(units={self.hidden_layer_units}, thetas_dims={self.thetas_dims}, ' \
            f'backcast_length={self.backcast_length}, ' \
            f'forecast_length={self.forecast_length}, share_thetas={self.share_thetas}, ' \
            f'dropout={self.dropout}, dropout_p={self.dropout_p}, neg_slope={self.neg_slope}) at @{id(self)}'

StableNBeatsNet:

In [ ]:
# Only the forward method is changed compared to standard NBeatsNet
class StableNBeatsNet(nn.Module): 
    
    def __init__(self, 
                 device,
                 backcast_length_multiplier,
                 forecast_length,
                 hidden_layer_units, thetas_dims, 
                 share_thetas,
                 nb_blocks_per_stack, n_stacks, share_weights_in_stack,
                 dropout = False, dropout_p = 0.0, 
                 neg_slope = 0.00):
        
        super().__init__()
        self.device = device
        self.backcast_length = backcast_length_multiplier * forecast_length
        self.forecast_length = forecast_length
        self.hidden_layer_units = hidden_layer_units
        self.thetas_dims = thetas_dims
        self.share_thetas = share_thetas
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.n_stacks = n_stacks
        self.share_weights_in_stack = share_weights_in_stack
        self.dropout = dropout
        self.dropout_p = dropout_p
        self.neg_slope = neg_slope
        
        self.stacks = []
        self.parameters = []
        
        print(f'| N-Beats')
        for stack_id in range(self.n_stacks):
            self.stacks.append(self.create_stack(stack_id))
        self.parameters = nn.ParameterList(self.parameters)
        
        
    def create_stack(self, stack_id):
        
        print(f'| --  Stack Generic (#{stack_id}) (share_weights_in_stack={self.share_weights_in_stack})')
        blocks = []
        for block_id in range(self.nb_blocks_per_stack):
            if self.share_weights_in_stack and block_id != 0:
                block = blocks[-1]  # pick up the last one when we share weights
            else:
                block = GenericNBeatsBlock(self.device,
                                           self.backcast_length,
                                           self.forecast_length,
                                           self.hidden_layer_units, self.thetas_dims, 
                                           self.share_thetas,
                                           self.dropout, self.dropout_p, 
                                           self.neg_slope)
                self.parameters.extend(block.parameters())
                print(f'     | -- {block}')
                blocks.append(block)
                
        return blocks

    
    def forward(self, backcast_arr):
        
        # dim backcast_arr = batch_size x shifts x backcast_length
        # shifts == 0 is standard input window, others are shifted lookback windows 
        # higher index = further back in time
        # feed different input windows (per batch) through the SAME network (check via list of learnable parameters)
        # see https://stackoverflow.com/questions/54444630/application-of-nn-linear-layer-in-pytorch-on-additional-dimentions
        
        forecast_arr = torch.zeros((backcast_arr.shape[0], # take batch size from backcast
                                    backcast_arr.shape[1], # take n of shifts from backcast
                                    self.forecast_length), dtype = torch.float).to(self.device)
        backcast_arr = backcast_arr.to(self.device)
        
        # loop through stacks (and blocks)
        for stack_id in range(len(self.stacks)):
            for block_id in range(len(self.stacks[stack_id])):
                b, f = self.stacks[stack_id][block_id](backcast_arr)
                backcast_arr = backcast_arr - b
                forecast_arr = forecast_arr + f  
                
        return backcast_arr, forecast_arr


# Training and evaluation

In [ ]:
def seed_torch(seed = 5101992):
    
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed) 

In [ ]:
class StableNBeatsLearner:
    
    def __init__(self,
                 device,
                 forecast_length,
                 configNBeats):
        
        gc.collect()
                
        self.device = device 
        self.forecast_length = forecast_length
        self.configNBeats = configNBeats
        
        if self.configNBeats["loss_function"] == 1:
            self.loss = RMSSE
        elif self.configNBeats["loss_function"] == 2:
            self.loss = RMSSE_m
        elif self.configNBeats["loss_function"] == 3:
            self.loss = SMAPE
        elif self.configNBeats["loss_function"] == 4:
            self.loss = MAPE
            
        self.rndseed = self.configNBeats["rndseed"]
        
        seed_torch(self.rndseed)
        
        print('--- Model ---')    
        self.model = StableNBeatsNet(self.device,
                                     self.configNBeats["backcast_length_multiplier"],
                                     self.forecast_length,
                                     self.configNBeats["hidden_layer_units"],
                                     self.configNBeats["thetas_dims"],
                                     self.configNBeats["share_thetas"],
                                     self.configNBeats["nb_blocks_per_stack"],
                                     self.configNBeats["n_stacks"],
                                     self.configNBeats["share_weights_in_stack"],                               
                                     self.configNBeats["dropout"],
                                     self.configNBeats["dropout_p"],
                                     self.configNBeats["neg_slope"])
        
        self.model = self.model.to(self.device)
        
        self.optim = torch.optim.Adam(self.model.parameters(), 
                                      lr = self.configNBeats["learning_rate"],
                                      weight_decay = self.configNBeats["weight_decay"])
        
        self.init_state = copy.deepcopy(self.model.state_dict())
        self.init_state_opt = copy.deepcopy(self.optim.state_dict())
        
        wandb.watch(self.model)
    
    
    def ts_padding(self, ts_train_data, ts_eval_data):
        
        # Some time series in the dataset are not long enough to support the specified:
        # forecast_length and backcast_length + backcast input shifts
        # we use zero padding for the time series that are too short (neutral effect on loss calculations)
        # + self.shifts comes from the number of extra observations needed to create the shifted inputs/targets
        # + (self.forgins - 1) comes from rolling origin evaluation
        
        length_train = (self.configNBeats["backcast_length_multiplier"] * self.forecast_length + 
                        self.forecast_length +
                        self.shifts)
        length_eval = (self.configNBeats["backcast_length_multiplier"] * self.forecast_length +
                       self.forecast_length +
                       self.shifts +
                       (self.forigins - 1))
        
        ts_train_pad = [x if x.size >= length_train else np.pad(x,
                                                                (int(length_train - x.size), 0), 
                                                                'constant', 
                                                                constant_values = 0) for x in ts_train_data]
        ts_eval_pad = [x if x.size >= length_eval else np.pad(x,
                                                              (int(length_eval - x.size), 0), 
                                                              'constant', 
                                                              constant_values = 0) for x in ts_eval_data]
        
        return ts_train_pad, ts_eval_pad
        
        
    def make_batch(self, batch_data, shuffle_origin = True):
        
        # If shuffle_origin = True --> batch for training --> random forecast origin based on LH 
        # If shuffle_origin = False --> batch for evaluation --> fixed forecast origin
        
        # Split the batch into input_list and target_list
        # In x_arr and target_arr: batch x shift x backcats_length/forecast_length
        x_arr = np.empty(shape = (len(batch_data), 
                                  self.shifts + 1,
                                  self.configNBeats["backcast_length_multiplier"] * self.forecast_length))
        target_arr = np.empty(shape = (len(batch_data), 
                                       self.shifts + 1, 
                                       self.forecast_length))
        
        # For every time series in the batch:
        # (1) slice the time series according to specific forecasting origin (depending on shuffle_origin)
        # (2) make shifted inputs/targets --> max number of shifts = forecast_length - 1
        # (3) fill x_arr and target_arr
        for j in range(len(batch_data)):
            i = batch_data[j]
            
            if shuffle_origin: 
                # suffle_origin --> only in training 
                
                ### --> also pick random scale --> does not result in improved results
                ### to remain as close as possible to nbeats paper: do not pick random scale
                ### i = i + i * np.random.default_rng().uniform(-0.95, 0.95, 1)
                
                # pick origin
                LH_max_offset = int(self.configNBeats["LH"] * self.forecast_length)
                ts_max_offset = int(len(i) -
                                    (self.configNBeats["backcast_length_multiplier"] * self.forecast_length + 
                                     self.forecast_length +
                                     self.shifts))
                max_offset = min(LH_max_offset, ts_max_offset)
                if max_offset < 1:
                    offset = np.zeros(1)
                else:
                    offset = np.random.randint(low = 0, high = max_offset)   
            else:
                offset = np.zeros(1)
            
            if offset == 0:
                for shift in range(self.shifts + 1):
                    if shift == 0:
                        x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length:-self.forecast_length]
                        target_arr[j, shift, :] = i[-self.forecast_length:]
                    else:
                        x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length-shift:-self.forecast_length-shift]
                        target_arr[j, shift, :] = i[-self.forecast_length-shift:-shift]
            else:
                for shift in range(self.shifts + 1):
                    x_arr[j, shift, :] = i[-self.forecast_length-self.configNBeats["backcast_length_multiplier"]*self.forecast_length-offset-shift:-self.forecast_length-offset-shift]
                    target_arr[j, shift, :] = i[-self.forecast_length-offset-shift:-offset-shift]
                    
        return x_arr, target_arr
                                        
                    
    def create_example_plots(self, output, target, actuals_train, final_evaluation = False):
        
        plot_forecasts = torch.cat((actuals_train, output))
        plot_actuals = torch.cat((actuals_train, target))
        random_sample_forecasts = plot_forecasts.squeeze()
        random_sample_actuals = plot_actuals.squeeze()
        x_axis = torch.arange(1, random_sample_forecasts.shape[0]+1)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x = x_axis.numpy(), y = random_sample_forecasts.numpy(),
                                 mode = 'lines+markers', name = 'forecasts'))
        fig.add_trace(go.Scatter(x = x_axis.numpy(), y = random_sample_actuals.numpy(),
                                 mode = 'lines+markers', name = 'actuals'))
        
        # We only visualize examples for last epoch
        if not final_evaluation:
            wandb.log({"example_plots_evaluation": fig})
        else:
            wandb.log({"example_plots_final_evaluation": fig})
            
            
    def evaluate(self, x_arr, target_arr,
                 epoch = None,
                 need_grad = True,
                 early_stop = False):
        
        losses = dict()
        
        # Inputs must be converted to np.array of Tensors (float)
        x_arr = torch.from_numpy(x_arr).float().to(self.device)
        target_arr = torch.from_numpy(target_arr).float().to(self.device)
        
        if need_grad:
            self.model.train()
            self.model.to(self.device)
            _, forecast_arr = self.model(x_arr)
            
            losses_forecast_shifts = 0.0
            for shift in range(self.shifts + 1):
                losses_forecast_shifts += self.loss(forecast_arr[:, shift, :], 
                                                    target_arr[:, shift, :], 
                                                    x_arr[:, shift, :])
            losses["forecast_accuracy"] = losses_forecast_shifts / (self.shifts + 1)
                        
            if self.shifts > 0:
                # dimensions = batch_size x shifted forecasts for stability computations
                forecast_base_arr = torch.zeros((forecast_arr.shape[0],
                                                 sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                dtype = torch.float).to(self.device)
                forecast_shift_arr = torch.zeros((forecast_arr.shape[0],
                                                  sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                 dtype = torch.float).to(self.device)
                col = 0
                for shift in range(1, self.shifts + 1):
                    for horizon_m1 in range(self.forecast_length - shift):
                        forecast_base_arr[:, col] = forecast_arr[:, 0, horizon_m1]
                        forecast_shift_arr[:, col] = forecast_arr[:, shift, horizon_m1 + shift]
                        col = col + 1
                losses["forecast_stability"] = self.loss(forecast_shift_arr, forecast_base_arr, x_arr[:, 0, :])
            else:
                losses["forecast_stability"] = torch.zeros(1)
                
        else:
            with torch.no_grad():
                self.model.eval()
                self.model.to(self.device)
                _, forecast_arr = self.model(x_arr)
                
                losses_forecast_shifts = 0.0
                for shift in range(self.shifts + 1):
                    losses_forecast_shifts += self.loss(forecast_arr[:, shift, :], target_arr[:, shift, :], x_arr[:, shift, :])
                losses["forecast_accuracy"] = losses_forecast_shifts / (self.shifts + 1)
                
                if self.shifts > 0:
                    forecast_base_arr = torch.zeros((forecast_arr.shape[0],
                                                     sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                    dtype = torch.float).to(self.device)
                    forecast_shift_arr = torch.zeros((forecast_arr.shape[0],
                                                      sum(range(self.forecast_length - self.shifts, self.forecast_length))),
                                                     dtype = torch.float).to(self.device)
                    col = 0
                    for shift in range(1, self.shifts + 1):
                        for horizon_m1 in range(self.forecast_length - shift):
                            forecast_base_arr[:, col] = forecast_arr[:, 0, horizon_m1]
                            forecast_shift_arr[:, col] = forecast_arr[:, shift, horizon_m1 + shift]
                            col = col + 1
                    losses["forecast_stability"] = self.loss(forecast_base_arr, forecast_shift_arr, x_arr[:, 0, :])
                else:
                    losses["forecast_stability"] = torch.zeros(1)
                    
                if not self.disable_plot:
                    if early_stop:
                        # Plot validation examples - of standard/unshifted input - for last epoch before break
                        # This part of the evaluation function is only called after training has been forced to stop
                        self.create_example_plots(forecast_arr[0, 0, :], target_arr[0, 0, :], x_arr[0, 0, :])
                    else:
                        # Plot validation examples - of standard/unshifted input - for last epoch
                        # This part of the evaluation function is called after training has been completed
                        if (epoch == self.configNBeats["epochs"]):
                            self.create_example_plots(forecast_arr[0, 0, :], target_arr[0, 0, :], x_arr[0, 0, :])
        
        return losses
    
    
    # Training of net (training data can include validation data) + validation or testing
    def train_net(self,
                  ts_train_m4m,
                  ts_eval_m4m,
                  forigins,
                  validation = True,
                  validation_earlystop = False,
                  disable_plot = True):
        
        self.forigins = forigins
        self.shifts = self.configNBeats["shifts"]
        self.validation = validation
        self.validation_earlystop = validation_earlystop
        self.disable_plot = disable_plot
        #assert self.shifts < self.forecast_length # max allowed number of shifts is forecast_length - 1
        
        # Data preprocessing depends on backcast_length_multiplier
        ts_train_pad, ts_eval_pad = self.ts_padding(ts_train_m4m, ts_eval_m4m)
        ts_train_pad = np.array(ts_train_pad, dtype = object)
        ts_eval_pad = np.array(ts_eval_pad, dtype = object)
        
        print('--- Training ---')
        
        # Containers to save train/evaluation losses and parameters
        tloss_combined, tloss_forecast_accuracy, tloss_forecast_stability = [], [], []
        eloss_combined, eloss_forecast_accuracy, eloss_forecast_stability = [], [], []
        #params = []
        
        # Main training loop
        self.model.load_state_dict(self.init_state)
        self.optim.load_state_dict(self.init_state_opt)
            
        seed_torch(self.rndseed)
        # Initialize early stopping object
        if self.validation_earlystop:
            early_stopping = EarlyStopping(patience = self.configNBeats["patience"], verbose = True)
            
        for epoch in range(1, self.configNBeats["epochs"]+1):
            
            start_time = time()
            # Shuffle train data
            np.random.shuffle(ts_train_pad)
            # Determine number of batches per epoch
            num_batches = int(ts_train_pad.shape[0] / self.configNBeats["batch_size"])
            
            # Training per epoch
            avg_tloss_combined_epoch = 0.0
            avg_tloss_forecast_accuracy_epoch = 0.0
            avg_tloss_forecast_stability_epoch = 0.0
            
            for k in range(num_batches):
                
                batch = np.array(ts_train_pad[k*self.configNBeats["batch_size"]:(k+1)*self.configNBeats["batch_size"]])
                x_arr, target_arr = self.make_batch(batch, shuffle_origin = True)
                
                self.optim.zero_grad()
                losses_batch = self.evaluate(x_arr, target_arr,
                                             epoch, need_grad = True, 
                                             early_stop = False)
                if self.shifts > 0:
                    loss_combined = ((self.configNBeats["lambda"] * losses_batch["forecast_stability"]) +
                                     ((1 - self.configNBeats["lambda"]) * losses_batch["forecast_accuracy"]))
                else:
                    loss_combined = losses_batch["forecast_accuracy"]
                loss_combined.backward()
                self.optim.step()
                
                #params = self.model.parameters()
                #total_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
                #if (epoch == 1 or epoch == self.configNBeats["epochs"]) and k == 0:
                #    print('Epoch {}/{} \t n_learnable_pars={:.4f}'.format(
                #        epoch,
                #        self.configNBeats["epochs"],
                #        total_params))
                
                wandb.log({"tloss_comb_step": loss_combined,
                           "tloss_fcacc_step": losses_batch["forecast_accuracy"],
                           "tloss_fcstab_step": losses_batch["forecast_stability"]})
                
                avg_tloss_combined_epoch += (loss_combined / num_batches)
                avg_tloss_forecast_accuracy_epoch += (losses_batch["forecast_accuracy"] / num_batches)
                avg_tloss_forecast_stability_epoch += (losses_batch["forecast_stability"] / num_batches)
                
            if self.validation: # validation_full and validation_earlystop
                
                # Evaluation per epoch
                avg_eloss_combined_epoch = 0.0
                avg_eloss_forecast_accuracy_epoch = 0.0
                avg_eloss_forecast_stability_epoch = 0.0

                for forigin in range(self.forigins):

                    # Only one batch, but one batch per forecast origin
                    if forigin < self.forigins-1:
                        eval_data_subset = np.array([x[:(-18 + forigin + self.forecast_length)] for x in ts_eval_pad],
                                                   dtype = object)
                    else:
                        eval_data_subset = np.array([x for x in ts_eval_pad], dtype = object)
                    x_arr, target_arr = self.make_batch(eval_data_subset, shuffle_origin = False)
                    
                    losses_evaluation = self.evaluate(x_arr, target_arr,
                                                      epoch, need_grad = False,
                                                      early_stop = False)
                    if self.shifts > 0:
                        loss_combined = ((self.configNBeats["lambda"] * losses_evaluation["forecast_stability"]) +
                                         ((1 - self.configNBeats["lambda"]) * losses_evaluation["forecast_accuracy"]))
                    else:
                        loss_combined = losses_evaluation["forecast_accuracy"]

                    avg_eloss_combined_epoch += (loss_combined / self.forigins)
                    avg_eloss_forecast_accuracy_epoch += (losses_evaluation["forecast_accuracy"] / self.forigins)
                    avg_eloss_forecast_stability_epoch += (losses_evaluation["forecast_stability"] / self.forigins)
                
                elapsed_time = time() - start_time

                print('Epoch {}/{} \t tloss_combined={:.4f} \t eloss_combined={:.4f} \t time={:.2f}s'.format(
                    epoch,
                    self.configNBeats["epochs"],
                    avg_tloss_combined_epoch,
                    avg_eloss_combined_epoch,
                    elapsed_time))
                
                wandb.log({"epoch": epoch,
                           "tloss_comb_evol": avg_tloss_combined_epoch,
                           "tloss_fcacc_evol": avg_tloss_forecast_accuracy_epoch,
                           "tloss_fcstab_evol": avg_tloss_forecast_stability_epoch,
                           "eloss_comb_evol": avg_eloss_combined_epoch,
                           "eloss_fcacc_evol": avg_eloss_forecast_accuracy_epoch,
                           "eloss_fcstab_evol": avg_eloss_forecast_stability_epoch})
                
                if self.validation_earlystop: # validation_earlystop 

                    # early_stopping needs the average epoch validation loss to check if it has decreased, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(avg_eloss_combined_epoch, self.model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        # Load the last checkpoint with the best model
                        self.model.load_state_dict(torch.load('checkpoint.pt'))
                        # Produce plots for final epoch before break
                        if not self.disable_plot:
                            for forigin in range(self.forigins):
                                # Only one batch, but one batch per forecast origin
                                if forigin < self.forigins-1:
                                    eval_data_subset = np.array([x[:(-18 + forigin + self.forecast_length)] for x in ts_eval_pad],
                                                                dtype = object)
                                else:
                                    eval_data_subset = np.array([x for x in ts_eval_pad], dtype = object)
                                x_arr, target_arr = self.make_batch(eval_data_subset, shuffle_origin = False)

                                losses_evaluation = self.evaluate(x_arr, target_arr,
                                                                  epoch, need_grad = False,
                                                                  early_stop = True)
                        # Break loop over epochs
                        break
                    
            else: # testing
                
                elapsed_time = time() - start_time

                print('Epoch {}/{} \t tloss_combined={:.4f} \t time={:.2f}s'.format(
                    epoch,
                    self.configNBeats["epochs"],
                    avg_tloss_combined_epoch,
                    elapsed_time))
                
                wandb.log({"epoch": epoch,
                           "tloss_comb_evol": avg_tloss_combined_epoch,
                           "tloss_fcacc_evol": avg_tloss_forecast_accuracy_epoch,
                           "tloss_fcstab_evol": avg_tloss_forecast_stability_epoch})

        wandb.log({"tloss_comb": avg_tloss_combined_epoch,
                   "tloss_fcacc": avg_tloss_forecast_accuracy_epoch,
                   "tloss_fcstab": avg_tloss_forecast_stability_epoch})
        
        print('--- Training done ---')
        print('--- Final evaluation ---')
        
        print('--- M4 evaluation ---')
        
        # Containers to save actuals and forecasts
        actuals = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        forecasts = np.empty(shape = (len(ts_eval_pad), self.forigins, self.forecast_length)) # n_series, forigin, forecast_length
        
        # Forecasts for each origin in rolling_window
        for forigin in range(self.forigins):
            
            # Only one batch, but one batch per forecast origin
            if forigin < self.forigins-1:
                eval_data_subset = np.array([x[:(-18 + forigin + self.forecast_length)] for x in ts_eval_pad],
                                           dtype = object)
            else:
                eval_data_subset = np.array([x for x in ts_eval_pad], dtype = object)
            x_arr, target_arr = self.make_batch(eval_data_subset, shuffle_origin = False)
            
            # Produce forecasts for subset of test data
            x_arr = torch.from_numpy(x_arr).float().to(self.device)
            target_arr = torch.from_numpy(target_arr).float().to(self.device)
            with torch.no_grad():
                self.model.eval()
                self.model.to(self.device)
                _, forecast_arr = self.model(x_arr)
                
            x_arr = x_arr.cpu() 
            target_arr = target_arr.cpu()
            forecast_arr = forecast_arr.cpu()
                
            # Plot 10 random examples per origin - of standard/unshifted input
            sample_ids = np.random.randint(low = 0, high = int(x_arr.shape[0]), size = 10)
            for sample_id in sample_ids:
                self.create_example_plots(forecast_arr[sample_id, 0, :], 
                                          target_arr[sample_id, 0, :], 
                                          x_arr[sample_id, 0, :],
                                          final_evaluation = True)
                
            # Save to containers
            forecasts[:, forigin, :] = forecast_arr[:, 0, :]
            actuals[:, forigin, :] = target_arr[:, 0, :]
            
        # Compute accuracy sMAPE
        sMAPE = 200 * np.mean(np.abs(actuals - forecasts) / (np.abs(forecasts) + np.abs(actuals)))
        
        # Compute stability Total MAC
        if self.forecast_length == 6:
            weight = np.mean(actuals[:, [0, 6, 12], :], axis = (1, 2))
        elif self.forecast_length == 18:
            weight = np.mean(actuals[:, 0, :], axis = -1)
        forecasts_helper = np.full((actuals.shape[0], 
                                    self.forigins,
                                    (self.forecast_length - 1) + self.forigins), np.nan)
        # n_series x self.forigins x ((forecast_length - 1) + forigins)
        for forigin in range(self.forigins):
            forecasts_helper[:, forigin, forigin:(forigin + self.forecast_length)] = forecasts[:, forigin, :]
        MAC_mat = np.abs(np.diff(forecasts_helper, axis = 1))
        # n_series x (self.forigins - 1) x ((forecast_length - 1) + forigins)
        MAC_mat_adjust = np.delete(MAC_mat, [0, (self.forecast_length - 1) + self.forigins - 1], 2)
        # n_series x (self.forigins - 1) x ((forecast_length - 1) + forigins - 2)
        MAC = np.nanmean(MAC_mat_adjust, axis = 1)
        # n_series x ((forecast_length - 1) + forigins - 2)
        ItemMAC = np.mean(MAC, axis = 1) / weight
        TotalMAC = np.mean(ItemMAC) * 100
        
        print('sMAPE_m4m={:.4f} \t TotalMAC_m4m={:.4f}'.format(sMAPE, TotalMAC))
        
        wandb.log({"sMAPE_m4m": sMAPE,
                   "TotalMAC_m4m": TotalMAC})
        
        # n_series, forigin, forecast_length
        fc_colnames = [str(i) for i in range(1, self.forecast_length + 1)]
        
        actuals_np = actuals#.numpy()
        m,n,r = actuals_np.shape
        actuals_arr = np.column_stack((np.repeat(np.arange(m) + 1, n), 
                                       np.tile(np.arange(n) + 1, m),
                                       actuals_np.reshape(m*n, -1)))
        actuals_df = pd.DataFrame(actuals_arr, columns = ['item_id', 'fc_origin'] + fc_colnames)
        helper_col = ['actual'] * len(actuals_df)
        actuals_df['type'] = helper_col
        
        forecasts_np = forecasts#.numpy()
        m,n,r = forecasts_np.shape
        forecasts_arr = np.column_stack((np.repeat(np.arange(m) + 1, n), 
                                         np.tile(np.arange(n) + 1, m),
                                         forecasts_np.reshape(m*n, -1)))
        forecasts_df = pd.DataFrame(forecasts_arr, columns = ['item_id', 'fc_origin'] + fc_colnames)
        helper_col = ['forecast'] * len(forecasts_df)
        forecasts_df['type'] = helper_col
        
        output_df_m4m = pd.concat([actuals_df, forecasts_df])
         
        wandb.join()
        
        return output_df_m4m

# Run experiments

Setup:

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)

In [ ]:
wandb_project_name = 'nbeats_stability_m4_monthly'
job_type_name = 'test'
# one of:
# - 'test', 
# - 'validation_full' --> e.g., for lambda value tuning
# - 'validation_earlystop' --> for specifying number of epochs

hyperparameter_defaults = dict()
hyperparameter_defaults['epochs'] = 155 
hyperparameter_defaults['batch_size'] = 512
hyperparameter_defaults['nb_blocks_per_stack'] = 1
hyperparameter_defaults['thetas_dims'] = 256
hyperparameter_defaults['n_stacks'] = 20
hyperparameter_defaults['share_weights_in_stack'] = False
hyperparameter_defaults["backcast_length_multiplier"] = 4
hyperparameter_defaults['hidden_layer_units'] = 256
hyperparameter_defaults['share_thetas'] = False
hyperparameter_defaults["dropout"] = False
hyperparameter_defaults["dropout_p"] = 0.0
hyperparameter_defaults["neg_slope"] = 0.00
hyperparameter_defaults['learning_rate'] = 0.001
hyperparameter_defaults["weight_decay"] = 0.00
hyperparameter_defaults["LH"] = 10
hyperparameter_defaults["rndseed"] = 2000
hyperparameter_defaults["loss_function"] = 1 # 1 == RMSSE / 2 == RMSSE_m / 3 == SMAPE / 4 == MAPE
hyperparameter_defaults["shifts"] = 1
hyperparameter_defaults['patience'] = 2000 # Only affects 'validation_earlystop' runs
hyperparameter_defaults['lambda'] = 0.15 # Weight of forecast stability loss in loss_combined
# Note that lambda is defined in the code as the proportion of stability in total loss (relative terms)
# In the paper, lambda is defined in absolute terms
# So: lambda_paper = lambda_code/(1-lambda_code)

In [ ]:
if job_type_name == 'test':
    is_val = False
    do_earlystop = False
    m4_train, m4_eval = valset_m4m, testset_m4m
elif job_type_name == 'validation_full':
    is_val = True
    do_earlystop = False
    m4_train, m4_eval = trainset_m4m, valset_m4m
elif job_type_name == 'validation_earlystop':
    is_val = True
    do_earlystop = True
    m4_train, m4_eval = trainset_m4m, valset_m4m

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Run:

In [ ]:
def sweep_function():
    wandb.init(config = hyperparameter_defaults,
               project = wandb_project_name,
               job_type = job_type_name)
    config = wandb.config
    run_name = wandb.run.name
    
    # Initialize model
    StableNBeats_model = StableNBeatsLearner(device, 6, config)
    # Train & evaluate
    forecasts_df_m4m = StableNBeats_model.train_net(m4_train, m4_eval, 13, is_val, do_earlystop)
    # Save forecasts
    forecasts_df_m4m.to_csv('m4m_nbeats_stability_' + job_type_name + '_' + run_name + '.csv', index = False)
    #forecasts_df_m4m.to_csv('/content/drive/My Drive/Colab Notebooks/Sweeps/m4m_nbeats_stability_' + job_type_name + '_' + run_name + '.csv', index = False)

In [ ]:
sweep_config = {
    "name": "sweep",
    "method": "grid",
    "parameters": {
        "rndseed": {
            "values": [2000]
        },
        "weight_decay": {
            "values": [0.0]
        },
        "dropout_p": {
            "values": [0.0]
        },
        "lambda": {
            "values": [0.0]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project = wandb_project_name)

In [ ]:
wandb.agent(sweep_id, function = sweep_function)